# VIX Options Chain Analysis

This notebook loads the VIX options chain data, normalises its column names, and computes mid-prices and option deltas using the cleaned schema.

In [ ]:
from pathlib import Path
import math
import pandas as pd

DATA_DIR = Path('data')
chain_path = DATA_DIR / 'VIX_65d_Options_ChainData.csv'
underlying_path = DATA_DIR / 'VIX_65d_Options_UnderlyingData.csv'

chain = pd.read_csv(chain_path)
# Normalise the chain columns by removing excess whitespace and internal spaces.
chain.columns = (
    chain.columns
    .str.strip()
    .str.replace(r"\s+", "", regex=True)
)

required_columns = {
    'Strike', 'CallBid', 'CallAsk', 'PutBid', 'PutAsk', 'CallIVM', 'PutIVM'
}
# Accept either Expiry or T for the maturity representation.
if {'Expiry', 'T'} & set(chain.columns):
    required_columns.add('Expiry' if 'Expiry' in chain.columns else 'T')
else:
    raise ValueError('Missing required maturity column: expected `Expiry` or `T`.')

missing_columns = sorted(required_columns - set(chain.columns))
if missing_columns:
    raise ValueError(f'Missing required columns after renaming: {missing_columns}')

underlying = pd.read_csv(underlying_path)
underlying.columns = (
    underlying.columns
    .str.strip()
    .str.replace(r"\s+", "", regex=True)
)

chain.head()

In [ ]:
# Helper functions for the normal CDF and Black-76 deltas.
def norm_cdf(x):
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))


def black76_call_delta(forward, strike, vol, tau, r=0.0, q=0.0):
    if vol <= 0 or tau <= 0 or forward <= 0 or strike <= 0:
        return float('nan')
    d1 = (math.log(forward / strike) + (q - r + 0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    return math.exp(-q * tau) * norm_cdf(d1)


def black76_put_delta(forward, strike, vol, tau, r=0.0, q=0.0):
    call_delta = black76_call_delta(forward, strike, vol, tau, r=r, q=q)
    if math.isnan(call_delta):
        return float('nan')
    return call_delta - math.exp(-q * tau)


forward = float(underlying.loc[0, 'Future'])
r = float(underlying.loc[0, 'r']) if 'r' in underlying.columns else 0.0
q = float(underlying.loc[0, 'q']) if 'q' in underlying.columns else 0.0

chain = chain.assign(
    CallMid=lambda df: (df['CallBid'] + df['CallAsk']) / 2,
    PutMid=lambda df: (df['PutBid'] + df['PutAsk']) / 2,
)

# Convert volatilities from percentages to decimals if necessary.
vol_scale = 1.0
if chain['CallIVM'].max() > 3:  # heuristically determine if values are in percentages.
    vol_scale = 100.0

chain = chain.assign(
    CallDelta=lambda df: df.apply(
        lambda row: black76_call_delta(
            forward,
            row['Strike'],
            row['CallIVM'] / vol_scale,
            row.get('Expiry', row.get('T')),
            r=r,
            q=q,
        ),
        axis=1,
    ),
    PutDelta=lambda df: df.apply(
        lambda row: black76_put_delta(
            forward,
            row['Strike'],
            row['PutIVM'] / vol_scale,
            row.get('Expiry', row.get('T')),
            r=r,
            q=q,
        ),
        axis=1,
    ),
)

chain[['Strike', 'CallMid', 'PutMid', 'CallDelta', 'PutDelta']].head()